In [1]:
import os

%pwd

'c:\\Users\\roypa\\Documents\\University of Manchester\\Text Mining\\Testing\\rp_text_summarizer\\research'

In [2]:
os.chdir("../")

%pwd

'c:\\Users\\roypa\\Documents\\University of Manchester\\Text Mining\\Testing\\rp_text_summarizer'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [19]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directory

In [36]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_evalation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directory([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config

In [37]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [38]:
# model evaluation class

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split dataset into smaller batches"""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                        batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                        column_text="article",
                        column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                               padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"],
                                       attention_mask=inputs["attention_mask"],
                                       length_penalty=0.8, num_beams=8, max_length=128)
            
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        # compute and return the ROUGE scores
        return metric.compute()

    def evaluate(self):
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)

         #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)


In [40]:
try:
    config = ConfigurationManager()
    model_evaluation_config = ModelEvaluation(config.get_model_evalation_config())
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-03-13 20:12:49,562: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-03-13 20:12:49,573: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-13 20:12:49,575: INFO: common: directory artifacts created successfully]
[2024-03-13 20:12:49,578: INFO: common: directory artifacts/model_evaluation created successfully]


c:\Users\roypa\anaconda3\conda\envs\textS\Lib\site-packages\datasets\load.py:753: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 5/5 [02:41<00:00, 32.27s/it]

[2024-03-13 20:15:41,034: INFO: rouge_scorer: Using default tokenizer.]
